Capstone Project
IBDM Dataset

This capstone is divided into a 4-phase duration. In this document, the entire details of the 4-phase(weekly) duration are given.



### Phase 1

    1) Webscrape the provided URL  - IMDB dataset:

    https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1

    2) Store the entire data in two different CSV files as per the given fields:

    The first CSV file data contains :

    Sno, Movie Name, Director Name, Duration, year, ratings, Metascore

    Bifurcate the Director field into subfields as per the number of directors of the movie belongs to such as Director1, director2

    The second CSV file contains the following:

    Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

    Bifurcate the stars field into 4 subfields as per the number of stars worked in the movie such as star1, star2, star3, star4

    Bifurcate the genre into 3 subfields as per the number of genres the movie belongs to such as :

    Genre1, genre2, genre3

### Phase 2

    Make two tables and corresponding columns provided in the above CSV files in SQLite DBMS. Insert all data of each CSV file in each of the created tables. Now start querying the table(s) in the  SQL workbench / SQLite database :

    Table 1: Sno, MovieName, Director Name, Duration, genre, ratings

    1)  Display all the details of movies created by directors Christopher and Matt Reeves.

    2) Display all the details of movies with a duration of 140 minutes to 190 minutes.

    3) Display all details of movies with ratings above 7 in ascending order.

    4) Display all movie names in descending order.

    5) Display movie name starts with ‘P’ and their rating is greater than 7.



    Table 2: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

    1) Display all movie names with star Arnold Schwarzenegger in ascending order.

    2    2) Display all details of the movie with the highest number of votes.

    3) Display movie names with gross collections in descending order.

    4) Display the gross collection of movies with the star Arnold.

    5) Display all details of movies with comedy and action genres.



    Make subquery :

    1) Display all details from both tables where movie names are the same.

    2)   Display all movie names, Director, ratings, and gross collection where the genre is action.

    3)      Display all details from both tables with the highest gross collection.

    4)      Display all details from both tables with the highest ratings

    5)      Display all details from both tables with the lowest gross collection and lowest ratings



    Now once completed with queries in the SQLite database, then make the exact query solutions by using PANDAS SQL in the data frame.  Load the CSV data in a data frame and start making solutions for all the above 15 queries using PANDAS SQL. You may use concat or merge joins per the requirements basis to make 5 join queries.



### Phase 3

    1) Now make only 1 data frame of two CSV files using the join operation of pandas and start doing EDA.

    2) Do the complete EDA in detail to explore the insights of data and write detailed observations of each analysis.



### Phase 4

    1) Write the complete Machine learning code to make predictions of votes and gross collection. Use appropriate models on their label basis. Remember you need to make 2 different predictions: vote and gross collection.

    2) Apply all the best techniques of scaling, and hyperparameter tuning, and avoid underfitting or overfitting (bias/variance)

    3) At the end save the best model and convey on which basis you have chosen that model. 

In [1]:
#required libraries
from bs4 import BeautifulSoup   
import requests
from requests import get
from warnings import warn
from time import sleep
from random import randint
import numpy as np
import seaborn as sns
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import warnings
warnings.filterwarnings("ignore")
import time
import pandas as pd
import re

In [2]:
# Scrape from imdb.com.
# You have to use the below URL:
#https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1

# Sno, Movie Name, Director Name, Duration, year, ratings, Metascore

pages = np.arange(1, 1743, 50) # Increase for each 50 results  ;.
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}


#initialize empty lists to store the variables scraped
titles = []
DirectorName=[]
Duration=[]
years = []
imdb_ratings = []
metascores = []


for page in pages:

    #get request 
    response = get("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action"
                   +"&sort=user_rating,desc&"
                  + "start="
                  + str(page)
                  + "&ref_=adv_nxt", headers=headers)

    #throw warning for status codes that are not 200
    if response.status_code != 200:
           warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
    for item in movie_containers:
        DirectorName.append(','.join([x.text for x in d]) if (d := item.select('a[href*="_dr_"]')) else None)       

            
   #extract the 50 movies for that page
    for container in movie_containers:      
        #Movies
        if container.find('div', class_ = 'ratings-metascore') is not None:
            #title
            title = container.h3.a.text
            titles.append(title)
        elif container.find('div', class_ = 'ratings-metascore') is None:
            title = container.find('div', class_= 'lister-item-content').h3.a.text 
            titles.append(title)
        else:
            titles.append(None)         
        
        
        if container.h3.find('span', class_= 'lister-item-year text-muted unbold') is not None:
            #year released
            year = container.h3.find('span', class_= 'lister-item-year text-muted unbold').text # remove the parentheses around the year and make it an integer
            years.append(year)
        else:
            years.append(None) # each of the additional if clauses are to handle type None data, replacing it with an empty string so the arrays are of the same length at the end of the scraping

        if container.p.find('span', class_ = 'runtime') is not None:
            #runtime
            time = int(container.p.find('span', class_ = 'runtime').text.replace(" min", "")) # remove the minute word from the runtime and make it an integer
            Duration.append(time)
        else:
            Duration.append(None)

        if float(container.strong.text) is not None:
            #IMDB ratings
            imdb = float(container.strong.text) # non-standardized variable
            imdb_ratings.append(imdb)
        else:
            imdb_ratings.append(None)

        if container.find('span', class_ = 'metascore') is not None:
            #Metascore
            m_score = container.find('span', class_ = 'metascore') 
            metascores.append(','.join(m_score))
        else:
            metascores.append(None)

print(len(titles),len(DirectorName),len(Duration),len(years),len(imdb_ratings),len(metascores))

IMDB_df_1 = pd.DataFrame({'movie': titles,
                          'DirectorName':DirectorName,
                          'Duration': Duration,
                          'year': years,
                          'rating': imdb_ratings,                  
                         'metascore': metascores }                    
                           )                         
                    
IMDB_df_1.loc[:, 'year'] = IMDB_df_1['year'].str[-5:-1] # two more data transformations after scraping
IMDB_df_1.loc[:, 'year'] = pd.to_numeric(IMDB_df_1['year'])
IMDB_df_1

1749 1749 1749 1749 1749 1749


,movie,DirectorName,Duration,year,rating,metascore
0,The Dark Knight,Christopher Nolan,152,2008,9.0,84
1,The Lord of the Rings: The Return of the King,Peter Jackson,201,2003,9.0,94
2,Inception,Christopher Nolan,148,2010,8.8,74
3,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178,2001,8.8,92
4,The Lord of the Rings: The Two Towers,Peter Jackson,179,2002,8.8,87
...,...,...,...,...,...,...
1744,Radhe,Prabhu Deva,109,2021,1.9,None
1745,Race 3,Remo D'Souza,160,2018,1.9,None
1746,Angels Apocalypse,"Sean Cain,Enzo Zelocchi",85,2015,1.7,None
1747,Elk*rtuk,Keith English,125,2021,1.5,None


In [3]:
IMDB_df_1.head(10)

,movie,DirectorName,Duration,year,rating,metascore
0,The Dark Knight,Christopher Nolan,152,2008,9.0,84
1,The Lord of the Rings: The Return of the King,Peter Jackson,201,2003,9.0,94
2,Inception,Christopher Nolan,148,2010,8.8,74
3,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178,2001,8.8,92
4,The Lord of the Rings: The Two Towers,Peter Jackson,179,2002,8.8,87
5,The Matrix,"Lana Wachowski,Lilly Wachowski",136,1999,8.7,73
6,Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,124,1980,8.7,82
7,Soorarai Pottru,Sudha Kongara,153,2020,8.7,None
8,Star Wars,George Lucas,121,1977,8.6,90
9,Terminator 2: Judgment Day,James Cameron,137,1991,8.6,75


In [4]:
import pandas as pd

# new data frame with split value columns
new = IMDB_df_1["DirectorName"].str.split(",", n = 1, expand = True)
 
# making separate first name column from new data frame
IMDB_df_1["Director1"]= new[0]
 
# making separate last name column from new data frame
IMDB_df_1["Director2"]= new[1]
 
# Dropping old Name columns
IMDB_df_1.drop(columns =["DirectorName"], inplace = True)
 
# df display
IMDB_df_1

,movie,Duration,year,rating,metascore,Director1,Director2
0,The Dark Knight,152,2008,9.0,84,Christopher Nolan,None
1,The Lord of the Rings: The Return of the King,201,2003,9.0,94,Peter Jackson,None
2,Inception,148,2010,8.8,74,Christopher Nolan,None
3,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92,Peter Jackson,None
4,The Lord of the Rings: The Two Towers,179,2002,8.8,87,Peter Jackson,None
...,...,...,...,...,...,...,...
1744,Radhe,109,2021,1.9,None,Prabhu Deva,None
1745,Race 3,160,2018,1.9,None,Remo D'Souza,None
1746,Angels Apocalypse,85,2015,1.7,None,Sean Cain,Enzo Zelocchi
1747,Elk*rtuk,125,2021,1.5,None,Keith English,None


In [5]:
IMDB_df_1.to_csv('firstCSV_IMDB.csv',index = True)

# The second CSV file contains the following:

Movie Name, stars, votes, Genre, Gross collection, popularity, Certification


In [6]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn

#Movie Name, stars, votes, Genre, Gross collection, popularity, Certification
pages = np.arange(1, 1743, 50) # Increase for each 50 results  ;.
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}

#initialize empty lists 
titles = []
stars = []
Certification = []
genres = []
gross_collection = []
popularity =[]
votes = []

for page in pages:

    #get request 
    response = get("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action"
                   +"&sort=user_rating,desc&"
                  + "start="
                  + str(page)
                  + "&ref_=adv_nxt", headers=headers)

#     sleep(randint(8,15))

    #throw warning for status codes that are not 200
    if response.status_code != 200:
           warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')

    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
    for item in movie_containers:
        stars.append(','.join([x.text for x in s]) if (s := item.select('a[href*="_st_"]')) else None)   #Stars        

   #extract the 50 movies for that page
    for container in movie_containers:
        #Movie Name
        if container.find('div', class_ = 'ratings-metascore') is not None:
            #title
            title = container.h3.a.text
            titles.append(title)
        elif container.find('div', class_ = 'ratings-metascore') is None:
            title = container.find('div', class_= 'lister-item-content').h3.a.text 
            titles.append(title)
        else:
            titles.append(None)


        if container.p.find('span', class_ = 'certificate') is not None:
            #rating
            rating = container.p.find('span', class_= 'certificate').text
            Certification.append(rating)
        else:
            Certification.append("")

        if container.p.find('span', class_ = 'genre') is not None:
            #genre
            genre = container.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',') # remove the whitespace character, strip, and split to create an array of genres
            genres.append(','.join(genre))
        else:
            genres.append("")
            
        if container.find('span', attrs = {'name':'nv'})['data-value'] is not None:
            nv = container.find_all('span', attrs = {'name':'nv'})
            #Gross Collection
            gross= nv[1].text if len(nv) > 1 else ""
            gross= gross.replace("$", "").replace("M", "")
            gross_collection.append(gross)
        else:
            gross_collection.append(None)


        if container.find('span', attrs = {'name':'nv'})['data-value'] is not None:
            #Number of votes
            vote = int(container.find('span', attrs = {'name':'nv'})['data-value'])
            votes.append(vote)
        else:
            votes.append(None) 
        
        if container.find('span', class_='lister-item-index unbold text-primary') is not None:
            #Popularity
            popular = container.find('span', class_='lister-item-index unbold text-primary').text.strip('.')            
            popularity.append(popular)
        else:
            popularity.append(None)

print(len(titles),len(stars),len(votes),len(genres),len(gross_collection),len(popularity),len(Certification))

IMDB_df_2 = pd.DataFrame({'movie': titles,
                          'stars':stars,
                          'votes': votes,
                          'genre': genres,                          
                          'gross_collection':gross_collection,
                          'popularity':popularity,
                          'Certification': Certification,
                              }
                          )

IMDB_df_2['popularity'] = IMDB_df_2['popularity'].str.replace(',', '').astype(int)
IMDB_df_2

1749 1749 1749 1749 1749 1749 1749


,movie,stars,votes,genre,gross_collection,popularity,Certification
0,The Dark Knight,"Christian Bale,Heath Ledger,Aaron Eckhart,Mich...",2717546,"Action, Crime, Drama",534.86,1,UA
1,The Lord of the Rings: The Return of the King,"Elijah Wood,Viggo Mortensen,Ian McKellen,Orlan...",1886664,"Action, Adventure, Drama",377.85,2,U
2,Inception,"Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot ...",2412357,"Action, Adventure, Sci-Fi",292.58,3,UA
3,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean",1915366,"Action, Adventure, Drama",315.54,4,U
4,The Lord of the Rings: The Two Towers,"Elijah Wood,Ian McKellen,Viggo Mortensen,Orlan...",1702937,"Action, Adventure, Drama",342.55,5,UA
...,...,...,...,...,...,...,...
1744,Radhe,"Salman Khan,Disha Patani,Randeep Hooda,Jackie ...",178758,"Action, Crime, Thriller",,1745,UA
1745,Race 3,"Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline ...",47520,"Action, Crime, Thriller",1.69,1746,UA
1746,Angels Apocalypse,"Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,...",42912,"Action, Fantasy, Sci-Fi",,1747,
1747,Elk*rtuk,"Vivianne Bánovits,András Mózes,Barna Bokor,Gab...",39414,"Action, Crime, Drama",,1748,


In [7]:
IMDB_df_2

,movie,stars,votes,genre,gross_collection,popularity,Certification
0,The Dark Knight,"Christian Bale,Heath Ledger,Aaron Eckhart,Mich...",2717546,"Action, Crime, Drama",534.86,1,UA
1,The Lord of the Rings: The Return of the King,"Elijah Wood,Viggo Mortensen,Ian McKellen,Orlan...",1886664,"Action, Adventure, Drama",377.85,2,U
2,Inception,"Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot ...",2412357,"Action, Adventure, Sci-Fi",292.58,3,UA
3,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean",1915366,"Action, Adventure, Drama",315.54,4,U
4,The Lord of the Rings: The Two Towers,"Elijah Wood,Ian McKellen,Viggo Mortensen,Orlan...",1702937,"Action, Adventure, Drama",342.55,5,UA
...,...,...,...,...,...,...,...
1744,Radhe,"Salman Khan,Disha Patani,Randeep Hooda,Jackie ...",178758,"Action, Crime, Thriller",,1745,UA
1745,Race 3,"Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline ...",47520,"Action, Crime, Thriller",1.69,1746,UA
1746,Angels Apocalypse,"Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,...",42912,"Action, Fantasy, Sci-Fi",,1747,
1747,Elk*rtuk,"Vivianne Bánovits,András Mózes,Barna Bokor,Gab...",39414,"Action, Crime, Drama",,1748,


In [8]:
IMDB_df_2.head(10)

,movie,stars,votes,genre,gross_collection,popularity,Certification
0,The Dark Knight,"Christian Bale,Heath Ledger,Aaron Eckhart,Mich...",2717546,"Action, Crime, Drama",534.86,1,UA
1,The Lord of the Rings: The Return of the King,"Elijah Wood,Viggo Mortensen,Ian McKellen,Orlan...",1886664,"Action, Adventure, Drama",377.85,2,U
2,Inception,"Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot ...",2412357,"Action, Adventure, Sci-Fi",292.58,3,UA
3,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean",1915366,"Action, Adventure, Drama",315.54,4,U
4,The Lord of the Rings: The Two Towers,"Elijah Wood,Ian McKellen,Viggo Mortensen,Orlan...",1702937,"Action, Adventure, Drama",342.55,5,UA
5,The Matrix,"Keanu Reeves,Laurence Fishburne,Carrie-Anne Mo...",1957334,"Action, Sci-Fi",171.48,6,A
6,Star Wars: Episode V - The Empire Strikes Back,"Mark Hamill,Harrison Ford,Carrie Fisher,Billy ...",1320784,"Action, Adventure, Fantasy",290.48,7,UA
7,Soorarai Pottru,"Suriya,Paresh Rawal,Aparna Balamurali,Urvashi",119471,"Action, Drama",,8,U
8,Star Wars,"Mark Hamill,Harrison Ford,Carrie Fisher,Alec G...",1393251,"Action, Adventure, Fantasy",322.74,9,U
9,Terminator 2: Judgment Day,"Arnold Schwarzenegger,Linda Hamilton,Edward Fu...",1122780,"Action, Sci-Fi",204.84,10,A


In [9]:
import pandas as pd

# new data frame with split value columns
newStar = IMDB_df_2["stars"].str.split(",", expand = True)

# making separate star1 column from new data frame
IMDB_df_2["star1"]= newStar[0]

# making separate star2 column from new data frame
IMDB_df_2["star2"]= newStar[1] 

# making separate star3 column from new data frame
IMDB_df_2["star3"]= newStar[2]

# making separate star4 column from new data frame
IMDB_df_2["star4"]= newStar[3]


# new data frame with split value columns
newgenre = IMDB_df_2["genre"].str.split(",", expand = True)

# making separate genre1 column from new data frame
IMDB_df_2["genre1"]= newgenre[0]

# making separate genre2 column from new data frame
IMDB_df_2["genre2"]= newgenre[1]    

# making separate genre3 column from new data frame
IMDB_df_2["genre3"]= newgenre[2]


# Dropping old Name columns
IMDB_df_2.drop(columns =["stars"], inplace = True)
 
# Dropping old Name columns
IMDB_df_2.drop(columns =["genre"], inplace = True)
 
#df display
IMDB_df_2.fillna('', inplace=True)
IMDB_df_2

,movie,votes,gross_collection,popularity,Certification,star1,star2,star3,star4,genre1,genre2,genre3
0,The Dark Knight,2717546,534.86,1,UA,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Action,Crime,Drama
1,The Lord of the Rings: The Return of the King,1886664,377.85,2,U,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,Action,Adventure,Drama
2,Inception,2412357,292.58,3,UA,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Action,Adventure,Sci-Fi
3,The Lord of the Rings: The Fellowship of the Ring,1915366,315.54,4,U,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,Action,Adventure,Drama
4,The Lord of the Rings: The Two Towers,1702937,342.55,5,UA,Elijah Wood,Ian McKellen,Viggo Mortensen,Orlando Bloom,Action,Adventure,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...
1744,Radhe,178758,,1745,UA,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff,Action,Crime,Thriller
1745,Race 3,47520,1.69,1746,UA,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez,Action,Crime,Thriller
1746,Angels Apocalypse,42912,,1747,,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham,Action,Fantasy,Sci-Fi
1747,Elk*rtuk,39414,,1748,,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás,Action,Crime,Drama


In [10]:
IMDB_df_2.to_csv('secondCSV_IMDB.csv',index = True)